In [1]:
# importo las librerias que voy a necesitar

! pip install nltk
import pandas as pd
import numpy as np
import re
from ast import literal_eval
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import ast
from pandas import json_normalize
import os


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ETL STEAM GAMES

In [3]:
df = pd.read_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA EXTRACCION/S_games.csv")

In [4]:
df

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,0.0,767400.0,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,0.0,773570.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,"['Single-player', 'Steam Achievements']",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
32131,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,0.0,733530.0,Sacada
32132,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
32133,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,0.0,658870.0,"xropi,stev3ns"


In [5]:
# reviso las columnas del dataframe

df.columns.tolist()

['publisher',
 'genres',
 'app_name',
 'title',
 'url',
 'release_date',
 'tags',
 'reviews_url',
 'specs',
 'price',
 'early_access',
 'id',
 'developer']

In [6]:
#creo nuevo dataframe para las columnas que me interesan
Games = df[["id", "title", "genres", "release_date", "specs", "price"]]

#en el nuevo df renombro las columnas
Games = Games.rename(columns={'id': 'Id_juego', "title": "Nombre_juego", "genres": "Genero", "release_date": "Año_Lanzamiento","specs" : "Especificaciones", "price" : "Precio" })

# Elimina las filas que tienen valores nulos en alguna de las columnas
Games = Games.dropna()


In [7]:
Games

,Id_juego,Nombre_juego,Genero,Año_Lanzamiento,Especificaciones,Precio
0,761140.0,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018-01-04,['Single-player'],4.99
1,643980.0,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']",2018-01-04,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play
2,670290.0,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",2017-07-24,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play
3,767400.0,弹炸人2222,"['Action', 'Adventure', 'Casual']",2017-12-07,['Single-player'],0.99
5,772540.0,Battle Royale Trainer,"['Action', 'Adventure', 'Simulation']",2018-01-04,"['Single-player', 'Steam Achievements']",3.99
...,...,...,...,...,...,...
32129,745400.0,Kebab it Up!,"['Action', 'Adventure', 'Casual', 'Indie']",2018-01-04,"['Single-player', 'Steam Achievements', 'Steam...",1.99
32130,773640.0,Colony On Mars,"['Casual', 'Indie', 'Simulation', 'Strategy']",2018-01-04,"['Single-player', 'Steam Achievements']",1.99
32131,733530.0,LOGistICAL: South Africa,"['Casual', 'Indie', 'Strategy']",2018-01-04,"['Single-player', 'Steam Achievements', 'Steam...",4.99
32132,610660.0,Russian Roads,"['Indie', 'Racing', 'Simulation']",2018-01-04,"['Single-player', 'Steam Achievements', 'Steam...",1.99


In [8]:
Games.dtypes

Id_juego            float64
Nombre_juego         object
Genero               object
Año_Lanzamiento      object
Especificaciones     object
Precio               object
dtype: object

In [9]:
# Redondeo y convierto a tipo entero la columna 'Id_juego'
Games['Id_juego'] = Games['Id_juego'].round().astype(int)  

# Convierto la columna 'Genero' de cadenas a estructuras de datos (listas)
Games["Genero"]= Games["Genero"].apply(literal_eval)  

In [10]:
Games.dtypes

Id_juego             int32
Nombre_juego        object
Genero              object
Año_Lanzamiento     object
Especificaciones    object
Precio              object
dtype: object

In [11]:
# Para la columna Año_Lanzamiento extraigo solo el año

# Defino funcion para extraer el año del texto en la columna

def Años(texto):

    # Busco las coincidencias en el texto
    coincidencias = re.findall(r'\b\d{4}\b', texto)

    # Devuelvo la lista de coincidencias encontradas o None si no se encuentran años
    return int(coincidencias[0]) if coincidencias else None

In [12]:
# Aplico la función Años a la columna Año_Lanzamiento
Games["Año_Lanzamiento"] = Games["Año_Lanzamiento"].apply(Años)

# Elimino las filas con valores nulos en la columna Año_Lanzamiento
Games = Games.dropna(subset=["Año_Lanzamiento"] )

# Redondeo y convierto a entero la columna Año_Lanzamiento
Games['Año_Lanzamiento'] = Games['Año_Lanzamiento'].round().astype(int)

# Aplico literal_eval a la columna Especificaciones
Games["Especificaciones"]= Games["Especificaciones"].apply(literal_eval)

C:\Users\SANDRA\AppData\Local\Temp\ipykernel_2628\2058493442.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Games['Año_Lanzamiento'] = Games['Año_Lanzamiento'].round().astype(int)
C:\Users\SANDRA\AppData\Local\Temp\ipykernel_2628\2058493442.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Games["Especificaciones"]= Games["Especificaciones"].apply(literal_eval)


In [13]:
Games

,Id_juego,Nombre_juego,Genero,Año_Lanzamiento,Especificaciones,Precio
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018,[Single-player],4.99
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play
3,767400,弹炸人2222,"[Action, Adventure, Casual]",2017,[Single-player],0.99
5,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018,"[Single-player, Steam Achievements]",3.99
...,...,...,...,...,...,...
32129,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018,"[Single-player, Steam Achievements, Steam Cloud]",1.99
32130,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018,"[Single-player, Steam Achievements]",1.99
32131,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018,"[Single-player, Steam Achievements, Steam Clou...",4.99
32132,610660,Russian Roads,"[Indie, Racing, Simulation]",2018,"[Single-player, Steam Achievements, Steam Trad...",1.99


In [14]:
#En la columna Precio convierto los valores decimales en 1 si el juego es pago y en 0 si el juego es gratis

def Convertir(valor):
    try:
        # Si el valor es decimal
        float_valor = float(valor)
        return 1
    except ValueError:
        # Si la conversión a decimal falla, trata el valor como cadena
        return 0


In [15]:
#Aplico la funcion

Games["Precio"] = Games["Precio"].apply(Convertir)
Games

C:\Users\SANDRA\AppData\Local\Temp\ipykernel_2628\2375649455.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Games["Precio"] = Games["Precio"].apply(Convertir)


,Id_juego,Nombre_juego,Genero,Año_Lanzamiento,Especificaciones,Precio
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018,[Single-player],1
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018,"[Single-player, Multi-player, Online Multi-Pla...",0
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017,"[Single-player, Multi-player, Online Multi-Pla...",0
3,767400,弹炸人2222,"[Action, Adventure, Casual]",2017,[Single-player],1
5,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018,"[Single-player, Steam Achievements]",1
...,...,...,...,...,...,...
32129,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018,"[Single-player, Steam Achievements, Steam Cloud]",1
32130,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018,"[Single-player, Steam Achievements]",1
32131,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018,"[Single-player, Steam Achievements, Steam Clou...",1
32132,610660,Russian Roads,"[Indie, Racing, Simulation]",2018,"[Single-player, Steam Achievements, Steam Trad...",1


In [40]:
#guardo como csv
Games.to_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA ETL/Games_ETL.csv", index=True)

ETL USER_REVIEWS

In [17]:
df1 = pd.read_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1 RESPALDO/DATA EXTRACCION/revision_usuarios.csv")

In [18]:
df1

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [19]:
#creo un nuevo dataframe sin la columna user_url

Revision_usuarios = df1.drop(columns={"user_url"}, axis=1)

In [20]:
Revision_usuarios

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [21]:
#creo la funcion evaluacion para evaluar de manera segura expresiones literales en cadenas y manejar posibles errores 
#sin interrumpir la ejecución del programa

def evaluacion(x):
    try:
        # Verifica si x es una cadena (str). Si es así, se realiza la evaluación literal; 
        # de lo contrario, se devuelve una lista vacía.
        return ast.literal_eval(x) if isinstance(x, str) else []
    # En caso de que se produzca un error durante la evaluación, se devuelve una lista vacía
    except (SyntaxError, ValueError):
        return []

In [22]:
# Aplico la función a cada fila de la columna
Revision_usuarios['reviews'] = Revision_usuarios['reviews'].apply(evaluacion)

In [23]:
Revision_usuarios

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [24]:
# reviso la primera fila de la columna 'reviews' después de aplicar la función
Revision_usuarios['reviews'].iloc[0][0]

{'funny': '',
 'posted': 'Posted November 5, 2011.',
 'last_edited': '',
 'item_id': '1250',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}

In [25]:
# veo que reviews es una lista con varios diccionarios
# extraigo valores con funciones de las columnas que considero importantes

# Creo una función para extraer 'item_id' de una lista de diccionarios
def item_id(lista_diccionarios):
    return [diccionario.get('item_id', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

# Creo una función para extraer 'recommend' de una lista de diccionarios
def recommend(lista_diccionarios):
    return [diccionario.get('recommend', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

# Creo una función para extraer 'review' de una lista de diccionarios
def review(lista_diccionarios):
    return [diccionario.get('review', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

# Creo una función para extraer 'posted' de una lista de diccionarios
def posted(lista_diccionarios):
    return [diccionario.get('posted', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

# Creo un nuevo DataFrame con los resultados
df_resultado = pd.DataFrame()


In [26]:
# Creo nuevas columnas para cada clave en el nuevo DataFrame

df_resultado['item_id'] = Revision_usuarios['reviews'].apply(item_id)
df_resultado['recommend'] = Revision_usuarios['reviews'].apply(recommend)
df_resultado['review'] = Revision_usuarios['reviews'].apply(review)
df_resultado['posted'] = Revision_usuarios['reviews'].apply(posted)

In [27]:
# Concateno horizontalmente df_resultado al DataFrame Revision_usuarios
Revision_usuarios = pd.concat([Revision_usuarios, df_resultado], axis=1)

# Elimino la columna 'reviews' del DataFrame Revision_usuarios
Revision_usuarios = Revision_usuarios.drop(columns={'reviews'}, axis=1)

# Desglosa las listas en las columnas 'item_id', 'recommend', 'review', 'posted' en filas duplicadas
Revision_usuarios = Revision_usuarios.explode(['item_id', 'recommend', 'review', 'posted'])


In [28]:
Revision_usuarios

,user_id,item_id,recommend,review,posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,"Posted November 5, 2011."
0,76561197970982479,22200,True,It's unique and worth a playthrough.,"Posted July 15, 2011."
0,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,"Posted April 21, 2011."
1,js41637,251610,True,I know what you think when you see this title ...,"Posted June 24, 2014."
1,js41637,227300,True,For a simple (it's actually not all that simpl...,"Posted September 8, 2013."
...,...,...,...,...,...
25797,76561198312638244,70,True,a must have classic from steam definitely wort...,Posted July 10.
25797,76561198312638244,362890,True,this game is a perfect remake of the original ...,Posted July 8.
25798,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,Posted July 3.
25798,LydiaMorley,730,True,:D,Posted July 20.


In [29]:
# en la columna posted voy a dejar solo el año con una funcion

def año(fecha):
    
    #verifico que la fecha no se nula y sea un str y si encuentra año lo devuelva entero de otra forma None
    if pd.notna(fecha) and isinstance(fecha, str):
        match = re.search(r'\b\d{4}\b', fecha)
        return int(match.group()) if match else None
    return None

In [30]:
# Aplico la función anterior y creo una nueva columna posted
Revision_usuarios['posted'] = Revision_usuarios['posted'].apply(año)

# Elimino filas con valores nulos en la columna año
Revision_usuarios = Revision_usuarios.dropna(subset=['posted'])

# Convertir la columna año a tipo entero
Revision_usuarios['posted'] = Revision_usuarios['posted'].astype(int)


C:\Users\SANDRA\AppData\Local\Temp\ipykernel_2628\40488797.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Revision_usuarios['posted'] = Revision_usuarios['posted'].astype(int)


In [31]:
Revision_usuarios

,user_id,item_id,recommend,review,posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011
0,76561197970982479,22200,True,It's unique and worth a playthrough.,2011
0,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
1,js41637,251610,True,I know what you think when you see this title ...,2014
1,js41637,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
25764,wayfeng,730,True,its FUNNNNNNNN,2015
25765,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015
25769,72947282842,730,True,Prettyy Mad Game,2015
25771,ApxLGhost,730,True,AMAZING GAME 10/10,2015


In [32]:
# de acuerdo a lo solicitado voy a crear el 'sentiment_analysis'

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\SANDRA\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [33]:
# Lleno los valores NaN con una cadena vacía
Revision_usuarios['review'] = Revision_usuarios['review'].fillna('')

# Inicio el analizador de sentimientos de VADER
sia = SentimentIntensityAnalyzer()

# Aplico análisis de sentimientos y asignar valores a una nueva columna
Revision_usuarios['sentiment_analysis'] = Revision_usuarios['review'].apply(lambda x: 2 if sia.polarity_scores(x)['compound'] >= 0.05 else (1 if -0.05 < sia.polarity_scores(x)['compound'] < 0.05 else 0))

# Elimino la columna original 'review'.
Revision_usuarios = Revision_usuarios.drop(columns={'review'}, axis=1)

C:\Users\SANDRA\AppData\Local\Temp\ipykernel_2628\947737565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Revision_usuarios['review'] = Revision_usuarios['review'].fillna('')
C:\Users\SANDRA\AppData\Local\Temp\ipykernel_2628\947737565.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Revision_usuarios['sentiment_analysis'] = Revision_usuarios['review'].apply(lambda x: 2 if sia.polarity_scores(x)['compound'] >= 0.05 else (1 if -0.05 < sia.polarity_scores(x)['compound'] < 0.05 else 0))


In [34]:
Revision_usuarios

,user_id,item_id,recommend,posted,sentiment_analysis
0,76561197970982479,1250,True,2011,2
0,76561197970982479,22200,True,2011,2
0,76561197970982479,43110,True,2011,2
1,js41637,251610,True,2014,2
1,js41637,227300,True,2013,2
...,...,...,...,...,...
25764,wayfeng,730,True,2015,1
25765,76561198251004808,253980,True,2015,2
25769,72947282842,730,True,2015,0
25771,ApxLGhost,730,True,2015,2


In [41]:
#guardo como csv
Revision_usuarios.to_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA ETL/Revision_usuarios_ETL.csv", index=True)

ETL USER_ITEMS

In [36]:
df2 = pd.read_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1 RESPALDO/DATA EXTRACCION/users_items.csv")

In [37]:
df2

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [42]:
#creo un nuevo df sin las columnas que no me interesan
User_items = df2.drop(columns={"items_count", "steam_id", "user_url" }, axis=1)


In [43]:
# Aplico la función evaluacion ya creada para convertir la cadena en una lista de diccionarios de manera segura
User_items['items'] = User_items['items'].apply(evaluacion)

In [44]:
User_items

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...
88305,76561198323066619,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,[]
88308,76561198329548331,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [45]:
User_items.dtypes

user_id    object
items      object
dtype: object

In [46]:
User_items['items'].iloc[0][0]

{'item_id': '10',
 'item_name': 'Counter-Strike',
 'playtime_forever': 6,
 'playtime_2weeks': 0}

In [47]:
# veo que items es una lista de diccionarios
# extraigo valores con funciones de los datos que me interesan

def item_id(lista_diccionarios):
    return [diccionario.get('item_id', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

def item_name(lista_diccionarios):
    return [diccionario.get('item_name', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

def playtime_forever(lista_diccionarios):
    return [diccionario.get('playtime_forever', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

# Creo un nuevo DataFrame con los resultados
df_resultado2 = pd.DataFrame()

In [48]:
# Creo nuevas columnas para cada clave en el nuevo DataFrame

df_resultado2['item_id'] = User_items['items'].apply(item_id)
df_resultado2['item_name'] = User_items['items'].apply(item_name)
df_resultado2['playtime_forever'] = User_items['items'].apply(playtime_forever)

In [49]:
# Concato el df_resultado2 con User_items

# combino las columnas de ambos DataFrames
User_items = pd.concat([User_items, df_resultado2], axis=1)

# Elimin0 la columna 'items' del DataFrame resultante
User_items = User_items.drop(columns={'items'}, axis=1)

In [50]:
User_items

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ..."
1,js41637,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ..."
2,evcentric,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507..."
3,Riot-Punch,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,..."
4,doctr,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47..."
...,...,...,...,...
88305,76561198323066619,"[413850, 413851, 413852, 413853, 413854, 41385...","[CS:GO Player Profiles, CS:GO Player Profiles:...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
88306,76561198326700687,"[11020, 6370, 13140, 51100, 107900, 113400, 20...","[TrackMania Nations Forever, Bloodline Champio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
88307,XxLaughingJackClown77xX,[],[],[]
88308,76561198329548331,"[304930, 227940, 346330, 373330, 388490, 52157...","[Unturned, Heroes & Generals, BrainBread 2, Al...","[677, 43, 0, 0, 3, 4, 3]"


In [51]:
# Uso el método explode para descomponer las listas en las columnas 'item_id', 'item_name' y 'playtime_forever'
# Esto se hace para expandir cada elemento de las listas en una nueva fila, manteniendo los valores de las otras columnas

User_items = User_items.explode(['item_id', 'item_name', 'playtime_forever'])

# Agrupa el DataFrame por las columnas 'user_id', 'item_id' y 'item_name', y luego suma los valores de 'playtime_forever' para cada grupo
User_items = User_items.groupby(['user_id', 'item_id', 'item_name'], as_index=False)['playtime_forever'].sum()


In [52]:
User_items

,user_id,item_id,item_name,playtime_forever
0,--000--,104700,Super Monday Night Combat,461
1,--000--,104900,ORION: Prelude,63
2,--000--,105600,Terraria,1172
3,--000--,113400,APB Reloaded,2763
4,--000--,1250,Killing Floor,2949
...,...,...,...,...
5094077,zzzmidmiss,94600,Hector: Ep 1,0
5094078,zzzmidmiss,94610,Hector: Ep 2,0
5094079,zzzmidmiss,94620,Hector: Ep 3,0
5094080,zzzmidmiss,9480,Saints Row 2,7


In [55]:
# convierto el archivo en csv y lo guardo, se comprime por el peso del archivo

User_items.to_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA ETL/User_Items_ETL.csv", index=True)